In [1]:
## package setup and import
import numpy as np
import datajoint as dj
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from pathlib import Path
from scipy.io import loadmat

In [2]:
## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [3]:
## ESTABLISH SCHEMA

schema = dj.schema('playground')
schema.drop(True)
schema = dj.schema('playground')

In [11]:
MainList.drop(True)
## Manual entry table
@schema
class MainList(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : char(3)
    datestr : char(8)
    id : char(3)
    ---
       
    count : int
    
    """

Tables dropped.  Restart kernel.


In [12]:
MainList()
MainList

__main__.MainList

In [13]:
items =[{'mouse_id' : 'JB1', 'id' : '1', 'datestr' : '20201001', 'count': 1},\
        {'mouse_id' : 'JB1', 'id' : '1', 'datestr' : '20201002', 'count': 2},\
        {'mouse_id' : 'JB1', 'id' : '2', 'datestr' : '20201002', 'count': 3},\
        {'mouse_id' : 'JB1', 'id' : '3', 'datestr' : '20201002', 'count': 4},\
        {'mouse_id' : 'JB1', 'id' : '2', 'datestr' : '20201001', 'count': 2},\
        {'mouse_id' : 'JB2', 'id' : '1', 'datestr' : '20201001', 'count': 5},\
        {'mouse_id' : 'JB2', 'id' : '2', 'datestr' : '20201001', 'count': 2},\
        {'mouse_id' : 'JB2', 'id' : '1', 'datestr' : '20201002', 'count': 3},\
        {'mouse_id' : 'JB3', 'id' : '1', 'datestr' : '20201001', 'count': 4},\
        {'mouse_id' : 'JB3', 'id' : '1', 'datestr' : '20201002', 'count': 7},\
        {'mouse_id' : 'JB3', 'id' : '1', 'datestr' : '20201003', 'count': 2},\
        {'mouse_id' : 'JB3', 'id' : '2', 'datestr' : '20201003', 'count': 14}]
items

[{'mouse_id': 'JB1', 'id': '1', 'datestr': '20201001', 'count': 1},
 {'mouse_id': 'JB1', 'id': '1', 'datestr': '20201002', 'count': 2},
 {'mouse_id': 'JB1', 'id': '2', 'datestr': '20201002', 'count': 3},
 {'mouse_id': 'JB1', 'id': '3', 'datestr': '20201002', 'count': 4},
 {'mouse_id': 'JB1', 'id': '2', 'datestr': '20201001', 'count': 2},
 {'mouse_id': 'JB2', 'id': '1', 'datestr': '20201001', 'count': 5},
 {'mouse_id': 'JB2', 'id': '2', 'datestr': '20201001', 'count': 2},
 {'mouse_id': 'JB2', 'id': '1', 'datestr': '20201002', 'count': 3},
 {'mouse_id': 'JB3', 'id': '1', 'datestr': '20201001', 'count': 4},
 {'mouse_id': 'JB3', 'id': '1', 'datestr': '20201002', 'count': 7},
 {'mouse_id': 'JB3', 'id': '1', 'datestr': '20201003', 'count': 2},
 {'mouse_id': 'JB3', 'id': '2', 'datestr': '20201003', 'count': 14}]

In [17]:
MainList.insert(items,skip_duplicates=True)
MainList()

mouse_id,datestr,id,count
JB1,20201001,1,1
JB1,20201001,2,2
JB1,20201002,1,2
JB1,20201002,2,3
JB1,20201002,3,4
JB2,20201001,1,5
JB2,20201001,2,2
JB2,20201002,1,3
JB3,20201001,1,4
JB3,20201002,1,7


In [18]:
itemsDf = pd.DataFrame(items)
itemsDf

,mouse_id,id,datestr,count
0,JB1,1,20201001,1
1,JB1,1,20201002,2
2,JB1,2,20201002,3
3,JB1,3,20201002,4
4,JB1,2,20201001,2
5,JB2,1,20201001,5
6,JB2,2,20201001,2
7,JB2,1,20201002,3
8,JB3,1,20201001,4
9,JB3,1,20201002,7


In [19]:
itemsCondense = itemsDf.groupby(['mouse_id','datestr']).size().reset_index().rename(columns={0:'items'})
itemsCondense

,mouse_id,datestr,items
0,JB1,20201001,2
1,JB1,20201002,3
2,JB2,20201001,2
3,JB2,20201002,1
4,JB3,20201001,1
5,JB3,20201002,1
6,JB3,20201003,2


In [21]:
itemsGroups = itemsCondense[['mouse_id','datestr']]
itemsGroups

,mouse_id,datestr
0,JB1,20201001
1,JB1,20201002
2,JB2,20201001
3,JB2,20201002
4,JB3,20201001
5,JB3,20201002
6,JB3,20201003


In [35]:
ItemGroups.drop(True)
# @schema
# class ItemGroups(dj.Manual):
#     definition = """
#     mouse_id : char(3)
#     datestr : char(8)
#     ---
#     items : int
#     """


@schema
class ItemGroups(dj.Manual):
    definition = """
    ->MainList.mouse_id
    ->MainList.datestr
    ---
    items : int
    """

ItemGroups()
ItemGroups.insert(itemsCondense,skip_duplicates=True)
ItemGroups()

Tables dropped.  Restart kernel.


DataJointError: Foreign key reference MainList.mouse_id could not be resolved

In [33]:
# DaySummary.drop(True)
@schema
class DaySummary(dj.Computed):
    definition = """
    ->ItemGroups
    ---
    average : int
    """
    
    def make
    


mouse_id,datestr,average


In [ ]:
DaySummary()